<a href="https://colab.research.google.com/github/JasonD28/CS-175/blob/master/model/faster_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 88kB/s 


In [2]:
%%shell
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi
cp PythonAPI/pycocotools /usr/local/lib/python3.6/dist-packages/ -r

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 12.77 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [3]:
%%shell
git clone https://github.com/JasonD28/CS-175.git
cd CS-175
cp model/dataset.py  ../

Cloning into 'CS-175'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 139 (delta 67), reused 51 (delta 18), pack-reused 0
Receiving objects: 100% (139/139), 932.04 KiB | 2.02 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [0]:
%%shell

# Using PyTorch helper functions to simplify training
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.5.1

cp references/detection/utils.py ../
cp references/detection/engine.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

In [0]:
import torch
import utils
from torch import optim
import torchvision
from dataset import PneumoniaDataset
from engine import train_one_epoch, evaluate
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

root = "drive/My Drive/cs-175-project/cs-175-data"

In [7]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
dataset = PneumoniaDataset(root, True)
test_dataset = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
test_dataset = torch.utils.data.Subset(test_dataset, indices[-50:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=6,
        collate_fn=utils.collate_fn)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
lr_step = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)


In [14]:
for epoch in range(3):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=200)
    lr_step.step()
    torch.save(model.state_dict(), 'drive/My Drive/cs-175-project/model_checkpoints/first_try_model.pt')
    evaluate(model, test_data_loader, device=device)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Epoch: [0]  [    0/13317]  eta: 3:36:03  lr: 0.000002  loss: 0.3285 (0.3285)  loss_classifier: 0.0081 (0.0081)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.2538 (0.2538)  loss_rpn_box_reg: 0.0666 (0.0666)  time: 0.9734  data: 0.6277  max mem: 2351
Epoch: [0]  [   10/13317]  eta: 1:14:20  lr: 0.000012  loss: 0.3285 (0.3353)  loss_classifier: 0.0081 (0.0053)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.2598 (0.2892)  loss_rpn_box_reg: 0.0354 (0.0408)  time: 0.3352  data: 0.0622  max mem: 2351


KeyboardInterrupt: ignored